**Algoritmo Evolution Strategis em Python**

**Autor**: Iran Freitas Ribeiro

**Disciplina**: Computação Natural

**Professor**: Renato A. Krohling

Implementação baseada no artigo: [Evolution strategies – A comprehensive introduction](https://link.springer.com/article/10.1023/A:1015059928466)

In [1]:
import numpy as np
from numpy import argsort
import functions as f
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import rand_score

In [2]:
def es_comma(data, n_clusters, bounds, n_iter, a, mu, lam, p, lograte=-1):
    """
    Executa o algoritmo de ES do tipo (mu, lambda)

    Params:
        - data: Dados originais
        - n_clusters: Número de clusters
        - bounds: limites dos espaço de busca
        - n_iter: número de gerações
        - a: força da mutação (step size)
        - mu: tamanho da população de pais
        - lam: tamanho da população de descendentes
        - p: numero de pais escolhidos para o processo de recombinação
        - lograte: taxa de exibição dos melhores resultados
    return:
        - best: melhor individuo
        - best_eval: score do melhor individuo
        - hist_scores: histórico de scores

    """

    # inicialização do melhor centroid e melhor score
    best, best_eval = None, 1e+10
    
    # inicia a população
    population = list(f.inicia_populacao(mu, bounds, n_clusters, data))
    hist_scores = []
    
    # loop principal do algoritmo
    for epoch in range(n_iter):        
        # cria lista de filhos para a geração epoch
        children = list()
        # cria lista de scores para os filhos
        scores_children = list()    
        for l in range(lam):
            ### Casamento, recombinação e mutação
            pais_marriage, ids_ = marriage(population, p)
            filho = recombination(pais_marriage)
            filho = mutation(filho,a)
            
            # calcula o score do filho atual
            scorefilho = f.eval_individual(filho, f.predict(data,filho), data)
            # atualiza o melhor individuo e melhor score
            if scorefilho < best_eval:
                best_eval = scorefilho
                best = filho
            # insere score do filho na lista de scores
            scores_children.append(scorefilho)
            # insere filho na lista de filhos
            children.append(filho)
        # ordena os scores do melhor para o maior
        ranks = argsort(argsort(scores_children))
        # seleciona os mu melhores filhos
        selected_children = [children[ranks[k]] for k,_ in enumerate(ranks[:mu])]
        # informa sobre a evolução do algoritmo
        if (lograte>0 and epoch%lograte==0):
            print('%d, Best: %.5f' % (epoch, best_eval))
        # atualiza a população
        population = selected_children
        # adiciona o melhor score no histórico de scores
        hist_scores.append(best_eval)
    return best, best_eval, hist_scores

In [3]:
def marriage(pais,p):
    """
    Escolhe os pais para o casamento
    
    Params:
        - pais: população de pais
        - p: numero de pais escolhidos para o casamento
    return:
        - pais_escolhidos: lista de pais escolhidos
        - indices_pais: indices dos pais escolhidos
    """
    # indice dos pais
    pais_ids = np.arange(0,len(pais))
    # escolha p idices aleatorioamente
    indices_pais = np.random.choice(pais_ids,size=p, replace=False)
    # seleciona os p pais referentes aos indices
    pais_escolhidos = [pais[i] for i in indices_pais]

    return pais_escolhidos, indices_pais

def recombination(pais):
    """
    Gera um filho a partir da recombinação de p pais

    Params:
        - pais: pais para serem recombinados

    Return:
        - filho: filho gerado a partir da recombinação
    """
    # cria um filho com 0 valores iguais à um pai
    filho = np.zeros_like(pais[0])
    p = len(pais)
    for i in range(len(filho)):
        # escolhe o pai
        j = np.random.randint(0,high=p)
        # escolhe a informação do pai que sera transmitida
        k = np.random.randint(0,filho.shape[0])
        # transmite a informação para o filho
        filho[i] = pais[j][k]
    return filho

def mutation(filho, a):
    """
    Aplica a recombinação no filho

    Params:
        - filho: individuo a ser recombinado
        - a: força aplicada na mutação (step size)
    
    return:
        - filho_: filho mutado

    """
    filho_ = filho + np.random.normal(0, a, size=filho.shape[1])
    return filho_

In [7]:
def experiments(n_testes, exp_name = "", args=[], save=False):
    """
    Realiza os experimentos

    Params:
        - n_testes: número de execuções
        - exp_name: nome do experimento
        - args: lista de argumentos
    
    return:
        - best_exp: melhor individuo 
        - best_score_exp: score do melhor individuo
        - best_historico: historico de scores do melhor individuo

    """
    ### definição das variáveis
    best_exp = None
    best_score_exp = np.inf
    best_historico = []
    list_bests = []
    list_scores = []
    rand_index = []

    for i in range(n_testes):
        X_treino, X_teste, y_treino, y_teste = train_test_split(args[0], args[1], test_size=0.2)
        
        # executa todo o algoritmo uma vez com os argumentos passados
        best, best_score, hist_scores = es_comma(X_treino,args[2], args[3], args[4], args[5], args[6], args[7], args[8])
        pred_labels = f.predict(X_teste, best)
        ri = rand_score(y_teste, pred_labels)
        rand_index.append(ri)

        ### salva informações da execução
        list_bests.append(best)
        list_scores.append(best_score)
        best_historico.append(hist_scores)
        print ("Rand index {:.4f}".format(ri))
    
    print ("Media rand_index {:.4f}".format(np.mean(rand_index)))            
    ### salva resultados para uso futuro

    if save:
        np.save("results/DE/rand_index_{}.npy".format(exp_name), np.array(rand_index))
        np.save('results/ES/best_historico_{}.npy'.format(exp_name), np.array(best_historico))
        np.save('results/ES/list_scores_{}.npy'.format(exp_name), np.array(list_scores))
        np.save('results/ES/best_exp_{}.npy'.format(exp_name),best_exp)
    return best_exp, best_historico

# Iris

In [4]:
iris_data, targets = f.load_iris()
labels_iris = iris_data[4].values
X_iris = iris_data[[0,1,2,3]].values

In [12]:
# definição dos parâmetros de execução
bounds = [0.1, 7.9]
n_clusters = 3
n_iter = 200
a = 0.015
mu = 40
lam = 110
p = 3
args_iris = [X_iris, labels_iris, n_clusters, bounds, n_iter, a, mu, lam, p]

In [13]:
best, list_hist = experiments(10, 'ES_iris', args_iris)

Rand index 0.9586
Rand index 0.8069
Rand index 0.7931
Rand index 0.9126
Rand index 0.8759
Rand index 0.9586
Rand index 0.8713
Rand index 0.9310
Rand index 0.8897
Rand index 0.9586
Media rand_index 0.8956


# Wine dataset

In [14]:
wine_data, wine_targets = f.load_wine()
labels_wine = wine_data[0].values
wine_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [15]:
# normalização dos dados
scaler_wine = MinMaxScaler()
wine_data_test= scaler_wine.fit_transform(wine_data[np.arange(1,14)])
X_wine = wine_data_test

In [16]:
# definição dos parâmetros
bounds = [0.0, 1.0]
n_clusters = 3
n_iter = 200
a = 0.015
mu = 40
lam = 110
p = 3
args_wine = [X_wine, labels_wine, n_clusters, bounds, n_iter, a, mu, lam, p]

In [17]:
n_tests = 10
best_wine, hist_wine = experiments(n_tests, 'ES_wine', args_wine)

Rand index 1.0000
Rand index 0.8571
Rand index 0.9635
Rand index 0.7968
Rand index 0.8349
Rand index 0.8937
Rand index 0.9603
Rand index 0.9365
Rand index 1.0000
Rand index 0.9238
Media rand_index 0.9167


# Breast Cancer Dataset

In [20]:
breast_cancer, targets_breast = f.load_breast_cancer()
colunas = breast_cancer.columns
# labels de cada linha da base de dados
labels_wine = breast_cancer['diagnosis'].values
# seleciona apenas as features reais e pega os valores
data_breast_cancer = breast_cancer[colunas[1:]]
X_data_breast = data_breast_cancer.values

In [21]:
# normalização dos dados
scaler_breast = MinMaxScaler()
X_data_breast = scaler_breast.fit_transform(X_data_breast)

In [26]:
# define range for input
bounds = [0.0, 1.0]
# define the total iterations
n_clusters = len(targets_breast)
n_iter = 2000
# define the maximum step size
a = 0.0015
#step_size = 0.01
# number of parents selected
mu = 40
# the number of children generated by parents
lam = 110
p = 3

In [27]:
args_breast = [X_data_breast, labels_wine, n_clusters, bounds, n_iter, a, mu, lam, p]
best_exp, best_historico = experiments(10, "ES_breast", args=args_breast)

Rand index 0.6261
Rand index 0.5082
Media rand_index 0.5671
